In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

            




In [2]:
driver = webdriver.Chrome()
driver.get("https://www.motogp.com/en/calendar")

# Wait for dynamic content to load if needed

sched_soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = sched_soup.find_all('ul', {'class': 'calendar-listings__month-listings'})

driver.quit()



In [3]:
schedule_details

[<ul class="calendar-listings__month-listings">
 <li class="calendar-listing__event-container js-event calendar-listing__event-container--finished" data-id="fc73530a-7d68-4a25-8d86-9cb932a3bca9" data-length="1" data-start="1701129600">
 <a class="calendar-listing__event" href="/en/calendar/2024/test/valencia-motogp-official-test-/fc73530a-7d68-4a25-8d86-9cb932a3bca9">
 <div class="calendar-listing__status-container">
 <div class="calendar-listing__status-type">
                             
                             
                             Test
                     </div>
 <div class="calendar-listing__status-bar">
 <div class="calendar-listing__status-icon"></div>
 <div class="calendar-listing__status-text">
                             finished
                         </div>
 </div>
 </div>
 <div class="calendar-listing__information">
 <div class="calendar-listing__details">
 <div class="calendar-listing__date-container">
 <div class="calendar-listing__date-start-container"

In [4]:
days = [] 
end_days = []
months = []
titles = [] 
descs = []
circuits = [] 
time = []
start_hours = []
start_minutes = []
end_hours = []
end_minutes = []

year = datetime.now().year

round_counter = 1


month_mapping = {
        'Jan': 1,
        'JAN': 1,
        'January': 1,
        'Feb': 2,
        'FEB': 2,
        'Februrary': 2,
        'Mar': 3,
        'MAR': 3,
        'March': 3,
        'Apr': 4,
        'APR': 4,
        'April': 4,
        'May': 5,
        'MAY': 5,
        'Jun': 6,
        'JUN': 6,
        'June': 6,
        'Jul': 7,
        'JUL': 7,
        'July': 7,
        'Aug': 8,
        'AUG': 8,
        'August': 8,
        'Sep': 9,
        'SEP': 9,
        'September': 9,
        'Oct': 10,
        'OCT': 10,
        'October': 10,
        'Nov': 11,
        'NOV': 11,
        'November': 11,
        'Dec': 12,
        'DEC': 12,
        'December': 12
    }



In [6]:
for i in schedule_details:
    events = i.find_all('a', {'class': 'calendar-listing__event'})
    for event in events:
        if 'Test' in event.find('div', {'class': "calendar-listing__title"}).text:
            pass
        else: 
            temp_desc_full = event.find('div', {'class': "calendar-listing__title"}).text.strip()
            temp_desc = temp_desc_full.split('\n')[0].strip()
#             descs are appended later once the lap count has been scraped from the 2nd page

            temp_circuit = event.find('div', {'class': 'calendar-listing__location-track-name'}).text.strip()
            circuits.append(temp_circuit)

           
            temp_title = 'MotoGP Round' + ' ' + str(round_counter)
            round_counter += 1
            titles.append(temp_title)
            
            link = event['href']
            
            driver = webdriver.Chrome()
            driver.get("https://www.motogp.com" + link)
            event_page = BeautifulSoup(driver.page_source, 'html.parser')
            wait = WebDriverWait(driver, 15)
            try:
                enable_all_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
                enable_all_button.click()
            except:
                # If clicking "Enable all" button fails, try to close the popup
                try:
                    close_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-close-btn-container")))
                    close_button.click()
                except:
                    # If both attempts fail, you may need to handle the popup in a different way
                    pass
            # Click the last 'li' with class "event-schedule__tab-list-item"
            tab = wait.until(EC.element_to_be_clickable((By.XPATH, '(//li[@class="event-schedule__tab-list-item"])[last()]')))
            tab.click()
            # Get the updated page source
            event_page = BeautifulSoup(driver.page_source, 'html.parser')
            driver.quit()
            
            day_tab = event_page.find('li', {'class':"event-schedule__tab-list-item event-schedule__tab-list-active"})
    
            temp_day = day_tab.find('div', {'class': "event-schedule__tab-label-day"}).text.strip()
            days.append(int(temp_day))
#             end_days will append after the time is scraped to account for events that end past midnight
            
            temp_month = day_tab.find('div', {'class': "event-schedule__tab-label-month"}).text.strip()
            converted_month = month_mapping.get(temp_month)
            months.append(converted_month)
           
            
            event_day_schedule = event_page.find('ul', {'class': 'event-schedule__content-list'})            
            for item in event_day_schedule.find_all('li', class_='event-schedule__content-item'):
                category = item.find('span', class_='event-schedule__content-category')
                name = item.find('div', class_='event-schedule__content-name')
                if category and name and category.text == 'MotoGP™' and name.text == 'Race':
                    laps = item.find('span', class_='event-schedule__content-lap').text
                    time = item.find('span', class_='event-schedule__content-time').text.split(':')
                    break

            
            descs.append(temp_desc + '\n' + laps)
            start_hour = int(time[0])
            start_hours.append(start_hour)
            start_minute = int(time[1])
            start_minutes.append(start_minute)
            
            end_hour = start_hour + 1
            if end_hour == 24:
                end_hours.append(0)
                end_days.append(int(temp_day) + 1)
            else:
                end_hours.append(end_hour)
                end_days.append(int(temp_day))
                end_minutes.append(start_minute)
            
            

            

In [7]:
days

[10, 24, 14, 28, 12, 26, 2, 16, 30, 7, 4, 18, 1, 8, 22, 29, 6, 19, 27, 3, 17]

In [8]:
end_days

[10, 24, 14, 28, 12, 26, 2, 16, 30, 7, 4, 18, 1, 8, 22, 29, 6, 20, 27, 3, 17]

In [9]:
months

[3, 3, 4, 4, 5, 5, 6, 6, 6, 7, 8, 8, 9, 9, 9, 9, 10, 10, 10, 11, 11]

In [10]:
titles

['MotoGP Round 1',
 'MotoGP Round 2',
 'MotoGP Round 3',
 'MotoGP Round 4',
 'MotoGP Round 5',
 'MotoGP Round 6',
 'MotoGP Round 7',
 'MotoGP Round 8',
 'MotoGP Round 9',
 'MotoGP Round 10',
 'MotoGP Round 11',
 'MotoGP Round 12',
 'MotoGP Round 13',
 'MotoGP Round 14',
 'MotoGP Round 15',
 'MotoGP Round 16',
 'MotoGP Round 17',
 'MotoGP Round 18',
 'MotoGP Round 19',
 'MotoGP Round 20',
 'MotoGP Round 21']

In [11]:
descs

['Qatar Airways Grand Prix of Qatar\n22 Laps',
 'Grande Prémio Tissot de Portugal\n25 Laps',
 'Red Bull Grand Prix of The Americas\n20 Laps',
 'Gran Premio de España\n25 Laps',
 'Grand Prix de France\n27 Laps',
 'Gran Premi Monster Energy de Catalunya\n24 Laps',
 "Gran Premio d'Italia\n23 Laps",
 'Grand Prix of Kazakhstan\n26 Laps',
 'Motul TT Assen\n26 Laps',
 'Liqui Moly Motorrad Grand Prix Deutschland\n30 Laps',
 'Monster Energy British Grand Prix\n20 Laps',
 'Motorrad Grand Prix von Österreich\n28 Laps',
 'Gran Premio de Aragón\n23 Laps',
 'Gran Premio Red Bull di San Marino e della Riviera di Rimini\n27 Laps',
 'Grand Prix of India\n22 Laps',
 'Pertamina Grand Prix of Indonesia\n27 Laps',
 'Motul Grand Prix of Japan\n24 Laps',
 'Australian Motorcycle Grand Prix\n27 Laps',
 'OR Thailand Grand Prix\n26 Laps',
 'PETRONAS Grand Prix of Malaysia\n20 Laps',
 'Gran Premio Motul de la Comunitat Valenciana\n27 Laps']

In [12]:
circuits

['Lusail International Circuit',
 'Autódromo Internacional do Algarve',
 'Circuit Of The Americas',
 'Circuito de Jerez - Ángel Nieto',
 'Le Mans',
 'Circuit de Barcelona-Catalunya',
 'Autodromo Internazionale del Mugello',
 'Sokol International Racetrack',
 'TT Circuit Assen',
 'Sachsenring',
 'Silverstone Circuit',
 'Red Bull Ring - Spielberg',
 'MotorLand Aragón',
 'Misano World Circuit Marco Simoncelli',
 'Buddh International Circuit',
 'Pertamina Mandalika Circuit',
 'Mobility Resort Motegi',
 'Phillip Island',
 'Chang International Circuit',
 'Petronas Sepang International Circuit',
 'Circuit Ricardo Tormo']

In [13]:
start_hours

[13, 10, 15, 8, 8, 8, 8, 4, 8, 8, 8, 8, 8, 8, 6, 3, 1, 23, 4, 2, 8]

In [14]:
start_minutes

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
end_hours

[14, 11, 16, 9, 9, 9, 9, 5, 9, 9, 9, 9, 9, 9, 7, 4, 2, 0, 5, 3, 9]

In [16]:
end_minutes

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [17]:
# driver = webdriver.Chrome()
# driver.get("https://www.motogp.com/en/calendar/2024/event/qatar/10a432bf-2496-4b2b-9bfa-19a555d9eee4")
# event_page = BeautifulSoup(driver.page_source, 'html.parser')
# event_page

In [18]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [19]:
MotoGP = Calendar()
MotoGP.add('prodid', '-//My calendar product//example.com//')
MotoGP.add('version', '2.0')
uid = 110

In [20]:
 
for title, circuit, day, month, desc, s_h, s_m, e_h, e_m, e_d in zip(titles, circuits, days, months, descs, start_hours, start_minutes, end_hours, end_minutes, end_days):
    ievent = Event()
    ievent.add('summary', title) #Title of the event
    ievent.add('description', desc)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(year, month, day, s_h, s_m, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtend', datetime(year, month, e_d, e_h, e_m, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtstamp', datetime(year, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
    ievent.add('location', circuit)
    ievent.add('uid', uid)
    uid += 1
    MotoGP.add_component(ievent)

In [21]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'MotoGP.ics'), 'wb')
f.write(MotoGP.to_ical())
f.close()

ics file will be generated at  C:\Users\chris\Documents\Calendars/
